<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL_EtOH_iPrOH_H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [41]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import numpy as np
from scipy.optimize import root,minimize

In [3]:
p=Props(['Ethanol','Isopropanol', 'Water'])

In [210]:
d=DotDict()

d.P = 1e5
d.N = 15
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.1,0.1, 0.8])
d.FT =320.
d.D = 0.15*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10


In [211]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [212]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_scaled_minmax = np.min(bp_scaled), np.max(bp_scaled)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled_minmax,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * (d.reflux_ratio+1)* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [213]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [214]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [215]:
eqs_wrapped = wrap(eqs)

In [216]:
sol=minimize(lambda x: 0., x, method='SLSQP', bounds=[(-15,15)]*len(x), constraints=dict(type='eq',fun=eqs_wrapped))

In [217]:
res=xtod(sol.x,d2)

In [218]:
eqs(xtod(sol.x,d2))

(array([ 1.23691279e-10, -1.60071068e-10, -1.20053301e-10, -3.20142135e-10,
        -7.27595761e-11, -9.45874490e-11, -8.00355338e-11,  3.63797881e-10,
        -1.89174898e-10,  2.54658516e-10, -1.81898940e-10,  1.20053301e-10,
        -2.00088834e-10,  2.83762347e-10,  2.21916707e-10,  5.09317033e-11,
        -4.80213203e-10, -1.74622983e-10, -1.45519152e-10, -6.40284270e-10,
        -1.96450856e-10, -6.54836185e-11, -3.49245965e-10, -1.09139364e-10,
         3.63797881e-11,  6.54836185e-11,  1.60071068e-10,  6.91215973e-11,
         1.09139364e-10, -1.38243195e-10,  8.36735126e-11,  3.71073838e-10,
        -2.11002771e-10,  4.36557457e-11,  7.27595761e-11,  2.18278728e-11,
         1.16415322e-10,  4.14729584e-10,  4.80213203e-10,  2.29192665e-10,
         1.45519152e-11,  2.91038305e-11,  4.00177669e-11,  2.91038305e-11,
         4.72937245e-10, -7.10542736e-15,  7.10542736e-15,  1.42108547e-14,
         0.00000000e+00, -2.84217094e-14, -4.97379915e-14,  0.00000000e+00,
         2.8

In [219]:
xtod(sol.x,d2)

{'B': 85.0,
 'D': 15.0,
 'F': 100.0,
 'FT': 320.0,
 'F_stage': 7,
 'Fz': array([0.1, 0.1, 0.8]),
 'L': array([150.        , 149.84947364, 149.69775798, 149.53451035,
        149.34255216, 149.08874914, 148.69217956, 256.6617169 ,
        256.66789079, 256.67449497, 256.68108469, 256.68511537,
        256.67288042, 256.56942814, 255.90039961]),
 'Lx': array([[0.33028896, 0.393412  , 0.27629904],
        [0.31220723, 0.40029727, 0.2874955 ],
        [0.29536889, 0.40261066, 0.30202045],
        [0.27886177, 0.39853196, 0.32260628],
        [0.26091083, 0.38408398, 0.35500519],
        [0.23677132, 0.3488851 , 0.41434359],
        [0.18436929, 0.25150598, 0.56412473],
        [0.18498354, 0.25095676, 0.5640597 ],
        [0.18564791, 0.25035194, 0.56400015],
        [0.18634984, 0.24965573, 0.56399443],
        [0.18700318, 0.2486971 , 0.56429973],
        [0.1871275 , 0.24659258, 0.56627991],
        [0.18405562, 0.23859109, 0.57735329],
        [0.16031938, 0.19562149, 0.64405913],
    